In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
sns.set()

In [2]:
foldername = "../../results_all/micro_bootstrapping_vm"

#foldername = "../../results_all/micro_bootstrapping_vm_opti"

In [3]:
variances = []

In [4]:
all_files = glob.glob(foldername + "/*")
for file in all_files:
    # Check if it's a csv file
    if (str(file).endswith(".csv") and str(file).find("summary") == -1):

        df_results = pd.read_csv(file, sep=";", names=['name','commit', "CI_min", "Median", "CI_max", 'jump','trend'], skiprows=1)
        df_results.sort_values(by=["commit"], inplace=True)

        nameparts=str(file).split("\\")
        name = nameparts[1]
        name = name[:-4]

        print(f"Plotting benchmark {name}")

        #Find % set y-limits
        min = -5
        max = 5
        for minSmall in df_results.CI_min.unique():
            if minSmall < min:
                min = minSmall

        for maxSmall in df_results.CI_max.unique():
            if maxSmall > max:
                max = maxSmall


        sns.set(style="whitegrid")
        plot, axes = plt.subplots(figsize=(15,9))
        axes.set_ylim([min, max])
        axes.set_xlim([0, 70])
        axes.set_xticks([0,10,20,30,40,50,60,70])

        uptrendMarker="$↗$"
        downtrendMarker="$↘$"
        jumpSize=1000
        trendSize=700
        fillColor="#bac7e6"
        edgeColor='black'


        #Plot CIs
        plot = sns.lineplot(data=df_results, x="commit", y="CI_min", color="royalblue", zorder=2)
        plot = sns.lineplot(data=df_results, x="commit", y="CI_max", color="royalblue", zorder=2)
        x_commits = df_results.commit
        plot.fill_between(x=x_commits, y1=df_results.CI_min, y2=df_results.CI_max, facecolor =fillColor, zorder=1)

        #Plot Median line
        plot = sns.lineplot(data=df_results, x="commit", y="Median", color="black")

        #PLot Up&Down Changes
        ups = df_results.loc[(df_results['jump'].astype(str).str.startswith("potential up", na=False))]
        plot = sns.regplot(data=ups,x="commit", y="Median", marker="^", fit_reg=False, scatter_kws={"color":"orange","alpha":1,"s":jumpSize, "zorder":3, 'edgecolor':edgeColor})
        ups = df_results.loc[(df_results['jump'].astype(str).str.startswith("definite up", na=False))]
        plot = sns.regplot(data=ups,x="commit", y="Median", marker="^", fit_reg=False, scatter_kws={"color":"red","alpha":1,"s":jumpSize, "zorder":3, 'edgecolor':edgeColor})

        ups = df_results.loc[(df_results['trend'].astype(str).str.startswith("potential up", na=False))]
        plot = sns.regplot(data=ups,x="commit", y="Median", marker=uptrendMarker, fit_reg=False, scatter_kws={"color":"orange","alpha":1,"s":trendSize, "zorder":3, 'edgecolor':edgeColor})
        ups = df_results.loc[(df_results['trend'].astype(str).str.startswith("definite up", na=False))]
        plot = sns.regplot(data=ups,x="commit", y="Median", marker=uptrendMarker, fit_reg=False, scatter_kws={"color":"red","alpha":1,"s":trendSize, "zorder":3, 'edgecolor':edgeColor})

        downs = df_results.loc[(df_results['jump'].astype(str).str.startswith("potential down", na=False))]
        plot = sns.regplot(data=downs,x="commit", y="Median", marker="v", fit_reg=False, scatter_kws={"color":"orange","alpha":1,"s":jumpSize, "zorder":3, 'edgecolor':edgeColor})
        downs = df_results.loc[(df_results['jump'].astype(str).str.startswith("definite down", na=False))]
        plot = sns.regplot(data=downs,x="commit", y="Median", marker="v", fit_reg=False, scatter_kws={"color":"red","alpha":1,"s":jumpSize, "zorder":3, 'edgecolor':edgeColor})

        downs = df_results.loc[(df_results['trend'].astype(str).str.startswith("potential down", na=False))]
        plot = sns.regplot(data=downs,x="commit", y="Median", marker=downtrendMarker, fit_reg=False, scatter_kws={"color":"orange","alpha":1,"s":trendSize, "zorder":3, 'edgecolor':edgeColor})
        downs = df_results.loc[(df_results['trend'].astype(str).str.startswith("definite down", na=False))]
        plot = sns.regplot(data=downs,x="commit", y="Median", marker=downtrendMarker, fit_reg=False, scatter_kws={"color":"red","alpha":1,"s":trendSize, "zorder":3, 'edgecolor':edgeColor})

        legend_elements = [
            Line2D([0], [0], color='black', lw=1, label='Median Performance Change'),
            mpatches.Patch(color=fillColor, label='99% Confidence Interval'),
            Line2D([0], [0], marker='^', color='w', label='Potential Performance Jump (Up)', markerfacecolor='orange', markeredgecolor=edgeColor, markersize=16),
            Line2D([0], [0], marker='v', color='w', label='Definite Performance Jump (Down)', markerfacecolor='red', markeredgecolor=edgeColor, markersize=16),
            Line2D([0], [0], marker=uptrendMarker, color='w', label='Potential Performance Trend (Up)', markerfacecolor='orange', markeredgecolor=edgeColor, markersize=16),
            Line2D([0], [0], marker=downtrendMarker, color='w', label='Definite Performance Trend (Down)', markerfacecolor='red', markeredgecolor=edgeColor, markersize=16)
        ]

        plot.legend(fontsize="medium", loc=1, title='', handles=legend_elements, labelspacing=0.8)

        plot.set(ylim=(min, max))
        plt.savefig(f'{foldername}/plots/{name}.png')
        plt.clf()

Plotting benchmark bench-fast-queue-throughput-concurrent-1000000
Plotting benchmark bench-fast-queue-throughput-serial-1000000
Plotting benchmark bench-index-db-add-tsids
Plotting benchmark bench-index-db-get-tsids
Plotting benchmark BenchmarkAdd-items_1000-2
Plotting benchmark BenchmarkAdd-items_10000-2
Plotting benchmark BenchmarkAdd-items_100000-2
Plotting benchmark BenchmarkAdd-items_1000000-2
Plotting benchmark BenchmarkAdd-items_10000000-2
Plotting benchmark BenchmarkAddMulti-items_1000-2
Plotting benchmark BenchmarkAddMulti-items_10000-2
Plotting benchmark BenchmarkAddMulti-items_100000-2
Plotting benchmark BenchmarkAddMulti-items_1000000-2
Plotting benchmark BenchmarkAddMulti-items_10000000-2
Plotting benchmark BenchmarkAppendDecimalToFloat-Integers-2
Plotting benchmark BenchmarkAppendDecimalToFloat-Ones-2
Plotting benchmark BenchmarkAppendDecimalToFloat-RealFloat-2
Plotting benchmark BenchmarkAppendDecimalToFloat-Zeros-2
Plotting benchmark BenchmarkAppendFloatToDecimal-Intege

<ipython-input-4-60ae2eb0cf78>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plot, axes = plt.subplots(figsize=(15,9))


<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

...

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>